In [ ]:
#clears solution dir
import os, shutil
folder = './solutions'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
#clears solution_organized dir
import os, shutil
folder = './solutions_organized'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
#checks reason for not completed problems
import os
import re

# Load and deduplicate problem names
nc_file = "not_completed/not_completed_problems.txt"
with open(nc_file, "r") as f:
    nc_problem_names = list(set(line.strip() for line in f))

print(f"Total unique not-completed problems: {len(nc_problem_names)}")

solutions_path = "solutions"

# Time thresholds and tolerance
timeout_thresholds = [3600, 7200]
tolerance = 5  # seconds

# Results
problems_out_of_time = []
problems_with_no_solution = []
problems_with_missing_output = []

for problem in nc_problem_names:
    problem_file = os.path.join(solutions_path, f"{problem}_output.sas")

    if not os.path.exists(problem_file):
        print(f"[MISSING] {problem} has no output file.")
        problems_with_missing_output.append(problem)
        continue

    with open(problem_file, "r") as f:
        lines = f.readlines()

    if not lines:
        print(f"[EMPTY] {problem} output file is empty.")
        problems_with_no_solution.append(problem)
        continue

    last_line = lines[-1].strip()
    match = re.search(r'Time taken: (\d+(?:\.\d+)?)', last_line)

    if match:
        time_taken = float(match.group(1))

        # Check if time_taken is close to any timeout threshold
        if any(abs(time_taken - threshold) <= tolerance for threshold in timeout_thresholds):
            problems_out_of_time.append(problem)
            print(f"[TIMEOUT] {problem}: {time_taken} seconds (within timeout window)")
        else:
            problems_with_no_solution.append(problem)
            print(f"[NO SOLUTION] {problem}: {time_taken} seconds (not near timeout)")
    else:
        print(f"[NO TIME INFO] {problem}: 'Time taken' not found.")
        problems_with_no_solution.append(problem)

# Summary
print(f"\nSummary:")
print(f"Problems out of time (~{timeout_thresholds}±{tolerance}s): {len(problems_out_of_time)}")
print(f"Problems with no solution: {len(problems_with_no_solution)}")
print(f"Problems with missing output file: {len(problems_with_missing_output)}")


: 

In [ ]:
#given a problem name find the corresponding logs in the log folder and print them

import os
import re
problem_name = "p020244_2"  # Example problem name
logs_folder = "logs"
pattern = re.compile(r'^logs_script_\d+_plan\{(?P<problem>[^}]+)\.pddl\}(?P<err>\.err)?\.txt$')

for fname in os.listdir(logs_folder):
    if pattern.match(fname):
        match = pattern.match(fname)
        if match.group("problem") == problem_name:
            with open(os.path.join(logs_folder, fname), 'r') as f:
                content = f.read()
                print(f"----------------------\nLog file: {fname}\nContent:\n{content}\n")
else:
    print(f"----------------\nFinished searching for logs related to problem: {problem_name}")

In [ ]:
#counts completed FULL PROBLEMS (Have .SOL file for all versions)
import os

solutions_path = "solutions"
completed_dict = {}
total_count_plans = 0
for file in os.listdir(solutions_path):
    if file.endswith(".SOL"):
        name = file.split("_")[0]
        if not (name in completed_dict):
            completed_dict[name] = 0
        completed_dict[name] += 1
        total_count_plans += 1

completed_count = 0
not_completed_count = 0
for key, val in completed_dict.items():
    if val >= 5:
        completed_count += 1
    else:
        # print(f"Problem: {key}, is NOT completed ({val}/5)")
        not_completed_count += 1

print("FULL PROBLEMS: ", completed_count)
print("NOT FULL PROBLEMS: ", not_completed_count)
print("TOTAL PLANS WITH SOL: ", total_count_plans)

In [ ]:
#copies original solutions to the solutions folder
import os
import shutil
solutions_folder = './solutions'
og_solutions_folder = './og_solutions'

for sol_file in os.listdir(og_solutions_folder):
    target_path = os.path.join(solutions_folder, f"{sol_file.split(".")[0]}_og.SOL")
    shutil.copy(sol_file, target_path)

In [ ]:
# Organizes solutions into a structured directory based on problem completeness
# It copies .SOL and .sas files for problems that have at least 5 solutions into a new directory structure. 
# todo we dont need all versions for each file, even just one is ok
import os
import re
import shutil

solutions_folder = './solutions'
target_sol_folder = './solutions_organized'

# Identify full problems based on having at least 6 .SOL files
completed_dict = {}
for file in os.listdir(solutions_folder):
    if file.endswith(".SOL"):
        problem = file.split(".")[0]
        if "_" in problem:
            problem = file.split("_")[0]
        completed_dict[problem] = completed_dict.get(problem, 0) + 1

full_problems = {problem for problem, cnt in completed_dict.items() if cnt == 6}
print("Full problems:", len(full_problems))

# Regex to match files like p123456_0.SOL or p123456_1.sas (case-insensitive)
pattern = re.compile(r'^(?P<problem>p\d+)_(\d+|og).*\.?(?P<ext>SOL|sas)$', re.IGNORECASE)

# Loop through all files in the solutions folder and copy only those for full problems
for filename in os.listdir(solutions_folder):
    file_path = os.path.join(solutions_folder, filename)
    if os.path.isfile(file_path):
        match = pattern.match(filename)
        if match:
            problem = match.group("problem")
            if problem in full_problems:
                ext = match.group("ext").lower()  # normalize extension to lower-case
                # Set target subdirectory based on file extension
                if ext == "sol":
                    subfolder = "sols"
                elif ext == "sas":
                    subfolder = "sas"
                else:
                    continue

                # Construct directory paths
                problem_folder = os.path.join(target_sol_folder, problem)
                target_folder = os.path.join(problem_folder, subfolder)
                os.makedirs(target_folder, exist_ok=True)

                # Build target file path and copy file
                target_file = os.path.join(target_folder, filename)
                # print(f"Copying {filename} to {target_file}")
                shutil.copy(file_path, target_file)


In [ ]:
#clears empty .sas coming from wrong pddls or aborted executions that dont end in error

import os
folder = './solutions'

files_to_remove = []
file_good = []
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    if file_path.endswith(".sas"):
        with open(file_path, 'r') as file:
            lines = file.readlines()
            file.close()
        if lines[0].startswith("Time"): # indicates that no steps were executed
            files_to_remove.append(file_path)
        else:
            file_good.append(file_path)
    
for file in files_to_remove:
    os.remove(file)
print (f"Removed {len(files_to_remove)} empty .sas files.")
print (f"Kept {len(file_good)} good .sas files.")

In [ ]:
#checks not completed problems if they have actually been processed, sometimes a problem goes NC because of an unrelated error or abort

import os

# Read and deduplicate problem names
nc_file = "not_completed/not_completed_problems.txt"
with open(nc_file, "r") as f:
    nc_problem_names = list(set(line.strip() for line in f))

print(f"Total unique problems listed: {len(nc_problem_names)}")

solutions_path = "./solutions"

# Check existence of files
problems_actually_nc = []
to_remove_already_have = []
to_remove_actually_solved = []
to_remove_never_processed = []

for problem in nc_problem_names:
    has_sas = os.path.exists(os.path.join(solutions_path, f"{problem}_output.sas"))
    has_sol = os.path.exists(os.path.join(solutions_path, f"{problem}.SOL"))

    if has_sas and not has_sol:
        problems_actually_nc.append(problem)
    elif has_sol and not has_sas:
        to_remove_already_have.append(problem)
    elif has_sas and has_sol:
        to_remove_actually_solved.append(problem)
    elif not has_sas and not has_sol:
        to_remove_never_processed.append(problem)    
    else:
        problems_actually_nc.append(problem)

# Output results
print(f"Problems with no matches (sas or sol), never processed: {len(to_remove_never_processed)}")
print(f"Problems with sol but no sas, to remove: {len(to_remove_already_have)}")
print(f"Problems with both sas and sol, to remove: {len(to_remove_actually_solved)}")
print(f"Problems that are actually not completed: {len(problems_actually_nc)}")

# to fix nc list write to file
# string_to_write =""

# for p in problems_actually_nc:
#     string_to_write+=f"{p}\n"

# with open(nc_file, "w") as f:
#     f.write(string_to_write.strip())

In [ ]:
# Clear older log files leaving only the newest .txt and .err.txt per problem

import os, re
from collections import defaultdict

logs_folder = "logs"

# Updated regex: the entire string inside { } becomes the problem name (e.g. "p34142_3")
pattern = re.compile(
    r'^logs_script_\d+_plan\{(?P<problem>[^}]+)\.pddl\}(?P<err>\.err)?\.txt$'
)

# Build a dict mapping each problem name to the list of associated log files.
problem_files = defaultdict(list)

for fname in os.listdir(logs_folder):
    m = pattern.match(fname)
    if m:
        problem = m.group("problem")
        problem_files[problem].append(fname)

# For each problem, determine the newest plain log file and keep its pair (.err.txt if exists)
files_to_keep = set()

for problem, files in problem_files.items():
    # Filter out plain log files (exclude those ending with ".err.txt")
    plain_logs = [f for f in files if not f.endswith(".err.txt")]
    if not plain_logs:
        # If no plain log exists, use the available files
        plain_logs = files
    # Pick the plain log file with the most recent modification time
    ref_file = plain_logs[0]
    newest_time = os.path.getctime(os.path.join(logs_folder, ref_file))
    for f in plain_logs[1:]:
        mtime = os.path.getctime(os.path.join(logs_folder, f))
        if mtime > newest_time:
            newest_time = mtime
            ref_file = f
    files_to_keep.add(ref_file)
    # If an error log exists for this run, keep it too.
    err_candidate = ref_file.replace(".txt", ".err.txt")
    if err_candidate in files:
        files_to_keep.add(err_candidate)
    # If the reference file is an error log, also consider its plain log.
    if ref_file.endswith(".err.txt"):
        plain_candidate = ref_file.replace(".err.txt", ".txt")
        if plain_candidate in files:
            files_to_keep.add(plain_candidate)

files_to_keep = list(files_to_keep)
print(f"Keeping {len(files_to_keep)} log files out of {len(os.listdir(logs_folder))} total files.")

# Remove files not in the newest set for each problem.
# for fname in os.listdir(logs_folder):
#     if pattern.match(fname) and fname not in files_to_keep:
#         os.remove(os.path.join(logs_folder, fname))
#         # print(f"Removed log file: {fname}")

In [ ]:
#take organized solution, create a class to represent a plan with initial state, actions and goals
#we have to start by iteratining on the organized solutions we have, and search the corrisponding problem pddl file in the original dataset folder to extract initial state
#actions can be taken from the .SOL file

#

import os
import re

class Plan:
    def __init__(self, problem_id, problem_full_name, initial_state, actions, goals, recognizability_class):
        self.problem_id = problem_id
        self.problem_full_name = problem_full_name
        self.initial_state = initial_state
        self.actions = actions
        self.goals = goals
        self.recognizability_class = recognizability_class

    def __repr__(self):
        return (f"Plan(problem_id={self.problem_id},\n"
                f"     problem_full_name={self.problem_full_name},\n"
                f"     initial_state={self.initial_state},\n"
                f"     actions={self.actions},\n"
                f"     goals={self.goals},\n"
                f"     recognizability_class={self.recognizability_class})")

def parse_pddl(pddl_file_path):
    """
    Parses a PDDL file to extract initial state and goals.
    This simple parser iterates through the file line by line.
    """
    initial_state = []
    goals = []
    mode = None
    try:
        with open(pddl_file_path, 'r') as f:
            for line in f:
                line_stripped = line.strip()
                # Start of initial state block
                if line_stripped.startswith("(:init"):
                    mode = "init"
                    # If there is content on the same line
                    remainder = line_stripped[len("(:init"):].strip()
                    if remainder and remainder != "(" and not remainder.startswith(";"):
                        initial_state.append(remainder)
                    continue
                # Start of goal block
                elif line_stripped.startswith("(:goal"):
                    mode = "goal"
                    remainder = line_stripped[len("(:goal"):].strip()
                    # Handle goal starting with (and ... if needed
                    if remainder.startswith("(and"):
                        remainder = remainder[len("(and"):].strip()
                        if remainder and remainder != "(" and not remainder.startswith(";"):
                            goals.append(remainder)
                    continue
                # End of a block
                elif line_stripped == ")" or line_stripped == ")))":
                    mode = None
                    continue

                # Append lines based on current block
                if mode == "init":
                    if line_stripped and not line_stripped.startswith(";"):
                        initial_state.append(line_stripped)
                elif mode == "goal":
                    if line_stripped and not line_stripped.startswith(";"):
                        goals.append(line_stripped)
    except Exception as e:
        print(f"Error parsing PDDL file {pddl_file_path}: {e}")
        raise Exception
    return initial_state, goals

def parse_sol(sol_file_path):
    """
    Parses a SOL file to extract the list of actions.
    Skips commented lines.
    """
    actions = []
    try:
        with open(sol_file_path, 'r') as f:
            for line in f:
                line_stripped = line.strip()
                # Skip comments (lines starting with ;)
                if line_stripped.startswith(";") or not line_stripped:
                    continue
                line_stripped = line_stripped.replace("(","").replace(")","")
                actions.append(line_stripped)
    except Exception as e:
        print(f"Error parsing SOL file {sol_file_path}: {e}")
    return actions

def main():
    # Base directories for organized solutions and dataset PDDL files.
    # Assumes the script is run from /home/rsignoroni/fastdownward_docker/
    solutions_organized_dir = "./solutions_organized"
    dataset_pddl_base = "./logistics/"  # adjust as needed

    plans = []

    # Iterate over each problem directory inside solutions_organized
    for problem in os.listdir(solutions_organized_dir):
        problem_dir = os.path.join(solutions_organized_dir, problem)
        if not os.path.isdir(problem_dir):
            continue

        sols_dir = os.path.join(problem_dir, "sols")
        if not os.path.exists(sols_dir):
            print(f"No 'sols' directory for problem {problem} in organized solutions.")
            continue

        # For simplicity, pick the first SOL file we find
        sol_files = [f for f in os.listdir(sols_dir) if f.upper().endswith(".SOL")]
        if not sol_files:
            print(f"No SOL files found in {sols_dir}")
            continue

        for sol_file in sol_files:
            sol_file_path = os.path.join(sols_dir, sol_file)
            actions = parse_sol(sol_file_path)

            # Determine the corresponding PDDL file.
            # Eg: logistics/0-0.2/p000146/p000146_0.pddl
            pddl_file_path = None
            problem_name = sol_file.split(".")[0]  # Extract the base problem name
            for folder in ["0-0.2", "0.2-0.4", "0.4-0.6", "0.6-0.8", "0.8-1.0"]:
                candidate_path = os.path.join(dataset_pddl_base, folder, problem, f"{problem_name}.pddl")
                if os.path.exists(candidate_path):
                    pddl_file_path = candidate_path
                    recognizability_class = folder
                break
            if pddl_file_path is None:
                print(f"PDDL file not found for problem {problem} in any subfolder.")
                continue

            initial_state, goals = parse_pddl(pddl_file_path)

            # Create a Plan object and add to the list.
            plan = Plan(problem_id=problem.split("_")[0], problem_version=problem, initial_state=initial_state, actions=actions, goals=goals, recognizability_class=recognizability_class)
            plans.append(plan)

    # Print out all collected plans
    for plan in plans:
        print(plan)
        print("=" * 40)

if __name__ == "__main__":
    main()

In [ ]:
#OLD
#used to rename original .SOL files to have _og.SOL suffix

import os
sol_folder = './solutions'

for file in os.listdir(sol_folder):
    if "_" not in file:
        name = file.split(".")[0]
        new_name = f"{name}_og.SOL"
        os.rename(os.path.join(sol_folder,file), os.path.join(sol_folder,new_name))

In [ ]:
#searches trough the sas files for the time taken by problems, and saves them
import os
import re

solutions_path = "solutions"
matching_problems = []

for filename in os.listdir(solutions_path):
    if not filename.endswith("_output.sas"):
        continue

    filepath = os.path.join(solutions_path, filename)

    with open(filepath, "r") as f:
        lines = f.readlines()

    if not lines:
        continue

    last_line = lines[-1].strip()
    match = re.search(r'Time taken: (\d+(?:\.\d+)?)', last_line)

    if match:
        time_taken = float(match.group(1))
        if 3600 < time_taken < 7200: #adjust as needed to filter times
            problem_name = filename.replace("_output.sas", "")
            matching_problems.append((problem_name, time_taken))

# Output results
print(f"Problems sorted by time taken: {len(matching_problems)}\n")
for problem, time_taken in sorted(matching_problems, key=lambda x: x[1]):
    print(f"{problem}: {time_taken:.2f} seconds")
    
#saves the problems to file
output_file = "test_results/matching_problems.txt"
with open(output_file, "w") as f:
    for problem, time_taken in sorted(matching_problems, key=lambda x: x[1]):
        f.write(f"{problem}: {time_taken:.2f} seconds\n")
